In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import os
from dotenv import load_dotenv
import requests
from src import insertar_datos_automatico, conexion_postgres, convertir_fechaeventos
import psycopg2
import datetime

In [2]:
df_lesiones = pd.read_csv("data/lesiones_completo_normalizado.csv")

In [3]:
df_lesiones.rename(columns = {
    'JUGADOR': 'Nombre_Jugador',
    'href': 'Nombre_Completo_Jugador',
    'equipo': 'Club',
    'TIPO': 'Tipo_Lesion',
    'SUB TIPO': 'Sub_Tipo_Lesion',
    'TIPO_ID': 'Tipo_Lesion_ID',
    'SUB_TIPO_ID': 'Sub_Tipo_Lesion_ID',
    'DATA': 'Fecha_incio_lesion',
    'FIM': 'Fecha_fin_lesion'
}, inplace = True)

In [4]:
df_lesiones = convertir_fechaeventos(df_lesiones)

In [5]:
df_lesiones = df_lesiones.dropna(subset=["playerID"])
df_lesiones["playerID"] = df_lesiones["playerID"].astype(int)
df_lesiones["clubID"] = df_lesiones["clubID"].astype(int)

In [6]:
df_lesiones.dtypes

Nombre_Jugador             object
Tipo_Lesion                object
Sub_Tipo_Lesion            object
Fecha_incio_lesion         object
Fecha_fin_lesion           object
Nombre_Completo_Jugador    object
Club                       object
playerID                    int64
clubID                      int64
Tipo_Lesion_ID              int64
Sub_Tipo_Lesion_ID          int64
dtype: object

In [7]:
df_lesiones = df_lesiones[df_lesiones["Tipo_Lesion"] == "Injury"]

In [8]:
df_lesiones.drop(columns=(["Tipo_Lesion_ID", "Tipo_Lesion"]), inplace=True)

In [12]:
df_lesiones.rename(columns = {
    'Sub_Tipo_Lesion': 'Tipo_Lesion',
    'Sub_Tipo_Lesion_ID': 'Tipo_Lesion_ID',
}, inplace = True)

In [13]:
tabla_indice_lesiones = df_lesiones[["Tipo_Lesion", "Tipo_Lesion_ID"]].drop_duplicates()

In [17]:
df_lesiones["ID_Lesion"] = df_lesiones["Nombre_Jugador"].index

In [19]:
load_dotenv()
dbname = os.getenv("dbname")
user = os.getenv("user")
password = os.getenv("password")
(conn,cur) = conexion_postgres(dbname, user, password)
insertar_datos_automatico(df_lesiones,"lesiones", cur, conn)
insertar_datos_automatico(tabla_indice_lesiones,"Lesiones_indice", cur, conn)

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)


In [20]:
df_lesiones.to_csv("data/Tabla_Lesiones.csv", index=False)
tabla_indice_lesiones.to_csv("data/Tabla_Lesiones_indice.csv", index=False)